<a href="https://colab.research.google.com/github/Ugonna244/nba-prediction/blob/main/nbaUntitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**##Predicting winner of 2022 NBA Championship using machine learning**

**Import the needed Libraries**

In [ ]:
# Load necessary libraries

import platform
import numpy as np
import pandas as pd
import sklearn as sk
import os

**Upload the training dataset**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving nba_elo.csv to nba_elo.csv


**Convert the dataset to a dataframe**

In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['nba_elo.csv']))

**Import necessary libraries for training**

In [ ]:
from sklearn.metrics import f1_score, make_scorer, classification_report

scorer = make_scorer(f1_score, pos_label = None, average = 'weighted')


**View top elements of the dataset**

In [ ]:
# Examine the dataset with its initial formatting
df.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,score1,score2,quality,importance,total_rating
0,1946-11-01,1947,0,NaN,TRH,NYK,1300.0,1300.0000,0.640065,0.359935,1293.2767,1306.7233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,68.0,0,NaN,NaN
1,1946-11-02,1947,0,NaN,CHS,NYK,1300.0,1306.7233,0.631101,0.368899,1309.6521,1297.0712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,47.0,0,NaN,NaN
2,1946-11-02,1947,0,NaN,DTF,WSC,1300.0,1300.0000,0.640065,0.359935,1279.6189,1320.3811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0,50.0,0,NaN,NaN
3,1946-11-02,1947,0,NaN,PRO,BOS,1300.0,1300.0000,0.640065,0.359935,1305.1542,1294.8458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,53.0,0,NaN,NaN
4,1946-11-02,1947,0,NaN,STB,PIT,1300.0,1300.0000,0.640065,0.359935,1304.6908,1295.3092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.0,51.0,0,NaN,NaN


**View the top 5 and bottom 5 of the dataframe for proper understanding**

In [ ]:
df

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,score1,score2,quality,importance,total_rating
0,1946-11-01,1947,0,NaN,TRH,NYK,1300.000000,1300.000000,0.640065,0.359935,1293.2767,1306.7233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,68.0,0,NaN,NaN
1,1946-11-02,1947,0,NaN,CHS,NYK,1300.000000,1306.723300,0.631101,0.368899,1309.6521,1297.0712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,47.0,0,NaN,NaN
2,1946-11-02,1947,0,NaN,DTF,WSC,1300.000000,1300.000000,0.640065,0.359935,1279.6189,1320.3811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0,50.0,0,NaN,NaN
3,1946-11-02,1947,0,NaN,PRO,BOS,1300.000000,1300.000000,0.640065,0.359935,1305.1542,1294.8458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,53.0,0,NaN,NaN
4,1946-11-02,1947,0,NaN,STB,PIT,1300.000000,1300.000000,0.640065,0.359935,1304.6908,1295.3092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.0,51.0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71945,2022-04-09,2022,0,NaN,MIN,CHI,1525.513746,1579.784297,0.565433,0.434567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1557.093688,1575.641044,0.678325,0.321675,NaN,NaN,76,29.0,53.0
71946,2022-04-09,2022,0,NaN,DEN,LAL,1540.948928,1481.797129,0.714257,0.285743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1596.551861,1521.953823,0.821170,0.178830,NaN,NaN,73,28.0,51.0
71947,2022-04-09,2022,0,NaN,PHO,SAC,1662.260235,1364.967937,0.907795,0.092205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1634.712133,1421.126858,0.859519,0.140481,NaN,NaN,59,3.0,31.0
71948,2022-04-09,2022,0,NaN,NOP,GSW,1449.249520,1655.076359,0.352244,0.647756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1520.080842,1612.149058,0.501471,0.498529,NaN,NaN,76,12.0,44.0


**View the data types we are dealing with**

In [ ]:
df.dtypes

date               object
season              int64
neutral             int64
playoff            object
team1              object
team2              object
elo1_pre          float64
elo2_pre          float64
elo_prob1         float64
elo_prob2         float64
elo1_post         float64
elo2_post         float64
carm-elo1_pre     float64
carm-elo2_pre     float64
carm-elo_prob1    float64
carm-elo_prob2    float64
carm-elo1_post    float64
carm-elo2_post    float64
raptor1_pre       float64
raptor2_pre       float64
raptor_prob1      float64
raptor_prob2      float64
score1            float64
score2            float64
quality             int64
importance        float64
total_rating      float64
dtype: object

**Parse in the date colomn**

In [ ]:
df = pd.read_csv('nba_elo.csv', parse_dates = ['date'])

**View the shape of your data**

In [ ]:
df.shape

(71950, 27)

**View the data types again**

In [ ]:
# View the updated dataset
df.dtypes

date              datetime64[ns]
season                     int64
neutral                    int64
playoff                   object
team1                     object
team2                     object
elo1_pre                 float64
elo2_pre                 float64
elo_prob1                float64
elo_prob2                float64
elo1_post                float64
elo2_post                float64
carm-elo1_pre            float64
carm-elo2_pre            float64
carm-elo_prob1           float64
carm-elo_prob2           float64
carm-elo1_post           float64
carm-elo2_post           float64
raptor1_pre              float64
raptor2_pre              float64
raptor_prob1             float64
raptor_prob2             float64
score1                   float64
score2                   float64
quality                    int64
importance               float64
total_rating             float64
dtype: object

**Rename the variables for easy understanding**

In [ ]:
# Correct column names
df.columns = ['Date', 'Season', 'neutral',  'playoff', 'Visitor Team', 'Home Team', 'elo1_pre', 
              'elo2_pre', 'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post', 'carm-elo1_pre', 'carm-elo2_pre', 'carm-elo_prob1' , 'carm-elo_prob2', 
              'carm-elo1_post', 'carm-elo2_post', 'raptor1_pre', 'raptor2_pre' , 'raptor_prob1',  'raptor_prob2','Visitor Score','Home Score' ,'quality' ,'importance','total_rating' ]

In [ ]:
df.shape

(71950, 27)

**View the changed colomns**

In [ ]:
df

,Date,Season,neutral,playoff,Visitor Team,Home Team,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,Visitor Score,Home Score,quality,importance,total_rating
0,1946-11-01,1947,0,NaN,TRH,NYK,1300.000000,1300.000000,0.640065,0.359935,1293.2767,1306.7233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,68.0,0,NaN,NaN
1,1946-11-02,1947,0,NaN,CHS,NYK,1300.000000,1306.723300,0.631101,0.368899,1309.6521,1297.0712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,47.0,0,NaN,NaN
2,1946-11-02,1947,0,NaN,DTF,WSC,1300.000000,1300.000000,0.640065,0.359935,1279.6189,1320.3811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0,50.0,0,NaN,NaN
3,1946-11-02,1947,0,NaN,PRO,BOS,1300.000000,1300.000000,0.640065,0.359935,1305.1542,1294.8458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,53.0,0,NaN,NaN
4,1946-11-02,1947,0,NaN,STB,PIT,1300.000000,1300.000000,0.640065,0.359935,1304.6908,1295.3092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.0,51.0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71945,2022-04-09,2022,0,NaN,MIN,CHI,1525.513746,1579.784297,0.565433,0.434567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1557.093688,1575.641044,0.678325,0.321675,NaN,NaN,76,29.0,53.0
71946,2022-04-09,2022,0,NaN,DEN,LAL,1540.948928,1481.797129,0.714257,0.285743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1596.551861,1521.953823,0.821170,0.178830,NaN,NaN,73,28.0,51.0
71947,2022-04-09,2022,0,NaN,PHO,SAC,1662.260235,1364.967937,0.907795,0.092205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1634.712133,1421.126858,0.859519,0.140481,NaN,NaN,59,3.0,31.0
71948,2022-04-09,2022,0,NaN,NOP,GSW,1449.249520,1655.076359,0.352244,0.647756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1520.080842,1612.149058,0.501471,0.498529,NaN,NaN,76,12.0,44.0


**Create a new colomn 'Home Win' which is the feature we are trying to predict**

In [ ]:
# Create new feature that notes whether the home team won
df['Home Win'] = df['Visitor Score'] < df['Home Score']

# Assign "class values" to the Home Win column

### This will also be the value that we want to predict ###
y_true = df['Home Win'].values

**View the dataframe again with the new colomn with the predicted value**

In [ ]:
df

,Date,Season,neutral,playoff,Visitor Team,Home Team,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,Visitor Score,Home Score,quality,importance,total_rating,Home Win
0,1946-11-01,1947,0,NaN,TRH,NYK,1300.000000,1300.000000,0.640065,0.359935,1293.2767,1306.7233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,68.0,0,NaN,NaN,True
1,1946-11-02,1947,0,NaN,CHS,NYK,1300.000000,1306.723300,0.631101,0.368899,1309.6521,1297.0712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,47.0,0,NaN,NaN,False
2,1946-11-02,1947,0,NaN,DTF,WSC,1300.000000,1300.000000,0.640065,0.359935,1279.6189,1320.3811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0,50.0,0,NaN,NaN,True
3,1946-11-02,1947,0,NaN,PRO,BOS,1300.000000,1300.000000,0.640065,0.359935,1305.1542,1294.8458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,53.0,0,NaN,NaN,False
4,1946-11-02,1947,0,NaN,STB,PIT,1300.000000,1300.000000,0.640065,0.359935,1304.6908,1295.3092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.0,51.0,0,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71945,2022-04-09,2022,0,NaN,MIN,CHI,1525.513746,1579.784297,0.565433,0.434567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1557.093688,1575.641044,0.678325,0.321675,NaN,NaN,76,29.0,53.0,False
71946,2022-04-09,2022,0,NaN,DEN,LAL,1540.948928,1481.797129,0.714257,0.285743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1596.551861,1521.953823,0.821170,0.178830,NaN,NaN,73,28.0,51.0,False
71947,2022-04-09,2022,0,NaN,PHO,SAC,1662.260235,1364.967937,0.907795,0.092205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1634.712133,1421.126858,0.859519,0.140481,NaN,NaN,59,3.0,31.0,False
71948,2022-04-09,2022,0,NaN,NOP,GSW,1449.249520,1655.076359,0.352244,0.647756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1520.080842,1612.149058,0.501471,0.498529,NaN,NaN,76,12.0,44.0,False


**View perfomance for home team games**

In [ ]:
# Establishing the baseline win percentage for the Home Teams overall
n_games = df['Home Win'].count()
n_homewins = df['Home Win'].sum()
win_percentage = n_homewins / n_games

print('Home Win percentage: {0:.2f}%'.format(100 * win_percentage))

NameError: ignored

**FI score for predicting baseline for Home Teams wins with simple classifier**

In [ ]:
# Predicting the baseline for Home Teams wins with simple classifier

from sklearn.metrics import f1_score

y_pred = [1] * len(y_true)
print('F1: {0:.4f}%'.format(f1_score(y_true, y_pred, pos_label = None, average = 'weighted') * 100))

F1: 20.9064%


**Determine the teams that would qualify for the 2021-2022 NBA PLAYOFFS by checking whether the home or visitor teams won their last games**

In [ ]:
# Determining whether the home and visitor teams won their last games
# This will update the Home Last Win & Visitor Last Win columns to either remain False or change to True

from collections import defaultdict

won_last = defaultdict(int)   # The default value of int is 0

# Inefficient method to deterime previous wins
for index, row in df.iterrows():
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    row['Home Last Win'] = won_last[home_team]
    row['Visitor Last Win'] = won_last[visitor_team]
    # Set current win
    won_last[home_team] = row['Home Win']
    won_last[visitor_team] = not row['Home Win']
df.loc[20:25]

,Date,Season,neutral,playoff,Visitor Team,Home Team,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,Visitor Score,Home Score,quality,importance,total_rating,Home Win
20,1946-11-13,1947,0,NaN,CHS,BOS,1329.3558,1275.2767,0.708260,0.291740,1334.7465,1269.8861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,61.0,0,NaN,NaN,False
21,1946-11-13,1947,0,NaN,DTF,PRO,1281.5840,1307.2230,0.605409,0.394591,1275.3634,1313.4436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,70.0,0,NaN,NaN,True
22,1946-11-14,1947,0,NaN,PHW,WSC,1304.6670,1320.6311,0.618633,0.381367,1308.6625,1316.6356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,65.0,0,NaN,NaN,False
23,1946-11-14,1947,0,NaN,STB,BOS,1305.7323,1269.8861,0.686110,0.313890,1308.4683,1267.1501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,62.0,0,NaN,NaN,False
24,1946-11-15,1947,0,NaN,TRH,PRO,1283.6519,1313.4436,0.599685,0.400315,1294.7554,1302.3401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,68.0,0,NaN,NaN,False
25,1946-11-16,1947,0,NaN,BOS,TRH,1267.1501,1294.7554,0.602702,0.397298,1271.9004,1290.0051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,49.0,0,NaN,NaN,False


**A view of team that won their home games and predicted to qualify for play offs**

In [ ]:
df.loc[90:95][['Date', 'Visitor Team', 'Visitor Score', 'Home Team', 'Home Score', 
               'Home Win',]]


,Date,Visitor Team,Visitor Score,Home Team,Home Score,Home Win
90,1946-12-12,PHW,49.0,WSC,64.0,True
91,1946-12-12,STB,68.0,CHS,88.0,True
92,1946-12-13,TRH,52.0,PIT,62.0,True
93,1946-12-14,BOS,77.0,PHW,65.0,False
94,1946-12-14,PRO,81.0,DTF,66.0,False
95,1946-12-14,WSC,72.0,CLR,52.0,False


**Set up basic Decision Tree Classifier**

In [ ]:
# Basic Decision Tree Classifier set up
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state = 14)

**Import metric for cross-validation performance**

In [ ]:
from sklearn.model_selection import cross_val_score

**Using elo ratings as a feature**

In [ ]:
# Use selected features as input for the classifier (target)
X_previouswins = df[['elo1_pre', 'elo2_pre']].values

# Decision Tree Classifier
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_previouswins, y_true, scoring = scorer)

# Print results
print("Using just the last result from the elo ratings")
print('F1: {0:.4f}%'.format(np.mean(scores) * 100))

Using just the last result from the elo ratings
F1: 58.7865%


**Using winning streak as a feature**

In [ ]:
# Taking into consideration winning streaks - What are the teams' win streaks coming into the game?
df["Home Win Streak"] = 0
df["Visitor Win Streak"] = 0

# Did the home and visitor teams win their last game?
from collections import defaultdict
win_streak = defaultdict(int)

for index, row in df.iterrows():  # Note that this is not the most efficient method
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["Home Win Streak"] = win_streak[home_team]
    row["Visitor Win Streak"] = win_streak[visitor_team]
    df.loc[index] = row    
    
    # Set current win streak
    if row["Home Win"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1

**View the added colomns**

In [ ]:
df.loc[50:60][['Date', 'Visitor Team', 'Visitor Score', 'Home Team', 'Home Score', 
               'Home Win', 'Home Win Streak', 'Visitor Win Streak']]

,Date,Visitor Team,Visitor Score,Home Team,Home Score,Home Win,Home Win Streak,Visitor Win Streak
50,1946-11-26,DTF,66.0,CLR,46.0,False,1,2
51,1946-11-26,PHW,66.0,BOS,54.0,False,0,0
52,1946-11-26,TRH,68.0,WSC,78.0,True,4,0
53,1946-11-27,NYK,67.0,STB,60.0,False,2,5
54,1946-11-27,PIT,62.0,CLR,60.0,False,0,0
55,1946-11-27,WSC,75.0,CHS,67.0,False,0,5
56,1946-11-28,BOS,59.0,PIT,55.0,False,1,0
57,1946-11-28,CHS,72.0,STB,75.0,True,0,0
58,1946-11-28,PHW,55.0,DTF,68.0,True,3,1
59,1946-11-28,PRO,58.0,NYK,60.0,True,6,1


**Using "elo1_pre",	"elo2_pre", "Home Win Streak", "Visitor Win Streak" as selected input features for the classifier (target)**

In [ ]:
# Use selected features as input for the classifier (target)
X_winstreak = df[["elo1_pre",	"elo2_pre", "Home Win Streak", "Visitor Win Streak"]].values

# Decision Tree Classifier
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_winstreak, y_true, scoring = scorer)

# Print results
print("Using elo ratings and winning as features")
print("F1: {0:.4f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
F1: 58.4556%


**Making inference using the test data set**

**Import the test set **bold text**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving nba_elo_latest.csv to nba_elo_latest.csv


**Convert dataset to dataframe**

In [ ]:
import io
test_df = pd.read_csv(io.BytesIO(uploaded['nba_elo_latest.csv']))

**View the test dataset**

In [ ]:
test_df

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,score1,score2,quality,importance,total_rating
0,2021-10-19,2022,0,NaN,MIL,BRK,1657.749890,1605.053732,0.706613,0.293387,1667.197782,1595.605840,NaN,NaN,NaN,NaN,NaN,NaN,1659.743630,1613.934913,0.700783,0.299217,127.0,104.0,92,14,53
1,2021-10-19,2022,0,NaN,LAL,GSW,1549.258851,1529.010908,0.666457,0.333543,1536.851828,1541.417931,NaN,NaN,NaN,NaN,NaN,NaN,1510.584522,1530.407615,0.608488,0.391512,114.0,121.0,59,54,57
2,2021-10-20,2022,0,NaN,CHO,IND,1438.333792,1487.419833,0.572751,0.427249,1441.652424,1484.101200,NaN,NaN,NaN,NaN,NaN,NaN,1457.196370,1532.967268,0.536682,0.463318,123.0,122.0,48,51,50
3,2021-10-20,2022,0,NaN,DET,CHI,1380.825861,1495.430593,0.478994,0.521006,1373.503545,1502.752909,NaN,NaN,NaN,NaN,NaN,NaN,1369.777383,1512.209592,0.442543,0.557457,88.0,94.0,26,19,23
4,2021-10-20,2022,0,NaN,TOR,WAS,1449.348147,1494.785854,0.577882,0.422118,1433.077337,1511.056663,NaN,NaN,NaN,NaN,NaN,NaN,1461.247869,1525.861517,0.546017,0.453983,83.0,98.0,48,66,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,2022-04-09,2022,0,NaN,MIN,CHI,1525.513746,1579.784297,0.565433,0.434567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1557.093688,1575.641044,0.678325,0.321675,NaN,NaN,76,29,53
1226,2022-04-09,2022,0,NaN,DEN,LAL,1540.948928,1481.797129,0.714257,0.285743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1596.551861,1521.953823,0.821170,0.178830,NaN,NaN,73,28,51
1227,2022-04-09,2022,0,NaN,PHO,SAC,1662.260235,1364.967937,0.907795,0.092205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1634.712133,1421.126858,0.859519,0.140481,NaN,NaN,59,3,31
1228,2022-04-09,2022,0,NaN,NOP,GSW,1449.249520,1655.076359,0.352244,0.647756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1520.080842,1612.149058,0.501471,0.498529,NaN,NaN,76,12,44


**View the test set data types**

In [ ]:
test_df.dtypes

date               object
season              int64
neutral             int64
playoff           float64
team1              object
team2              object
elo1_pre          float64
elo2_pre          float64
elo_prob1         float64
elo_prob2         float64
elo1_post         float64
elo2_post         float64
carm-elo1_pre     float64
carm-elo2_pre     float64
carm-elo_prob1    float64
carm-elo_prob2    float64
carm-elo1_post    float64
carm-elo2_post    float64
raptor1_pre       float64
raptor2_pre       float64
raptor_prob1      float64
raptor_prob2      float64
score1            float64
score2            float64
quality             int64
importance          int64
total_rating        int64
dtype: object

In [ ]:
test_df.columns = ['Date', 'Season', 'neutral',  'playoff', 'Visitor Team', 'Home Team', 'elo1_pre', 
              'elo2_pre', 'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post', 'carm-elo1_pre', 'carm-elo2_pre', 'carm-elo_prob1' , 'carm-elo_prob2', 
              'carm-elo1_post', 'carm-elo2_post', 'raptor1_pre', 'raptor2_pre' , 'raptor_prob1',  'raptor_prob2','Visitor Score','Home Score' ,'quality' ,'importance','total_rating' ]

**Rename the colomns for easy manipulation**

In [ ]:
# Correct column names
test_df.columns = ['Date', 'Season', 'neutral',  'playoff', 'Visitor Team', 'Home Team', 'elo1_pre', 
              'elo2_pre', 'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post', 'carm-elo1_pre', 'carm-elo2_pre', 'carm-elo_prob1' , 'carm-elo_prob2', 
              'carm-elo1_post', 'carm-elo2_post', 'raptor1_pre', 'raptor2_pre' , 'raptor_prob1',  'raptor_prob2','Visitor Score','Home Score' ,'quality' ,'importance','total_rating' ]

In [ ]:
test_df

,Date,Season,neutral,playoff,Visitor Team,Home Team,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,Visitor Score,Home Score,quality,importance,total_rating
0,2021-10-19,2022,0,NaN,MIL,BRK,1657.749890,1605.053732,0.706613,0.293387,1667.197782,1595.605840,NaN,NaN,NaN,NaN,NaN,NaN,1659.743630,1613.934913,0.700783,0.299217,127.0,104.0,92,14,53
1,2021-10-19,2022,0,NaN,LAL,GSW,1549.258851,1529.010908,0.666457,0.333543,1536.851828,1541.417931,NaN,NaN,NaN,NaN,NaN,NaN,1510.584522,1530.407615,0.608488,0.391512,114.0,121.0,59,54,57
2,2021-10-20,2022,0,NaN,CHO,IND,1438.333792,1487.419833,0.572751,0.427249,1441.652424,1484.101200,NaN,NaN,NaN,NaN,NaN,NaN,1457.196370,1532.967268,0.536682,0.463318,123.0,122.0,48,51,50
3,2021-10-20,2022,0,NaN,DET,CHI,1380.825861,1495.430593,0.478994,0.521006,1373.503545,1502.752909,NaN,NaN,NaN,NaN,NaN,NaN,1369.777383,1512.209592,0.442543,0.557457,88.0,94.0,26,19,23
4,2021-10-20,2022,0,NaN,TOR,WAS,1449.348147,1494.785854,0.577882,0.422118,1433.077337,1511.056663,NaN,NaN,NaN,NaN,NaN,NaN,1461.247869,1525.861517,0.546017,0.453983,83.0,98.0,48,66,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,2022-04-09,2022,0,NaN,MIN,CHI,1525.513746,1579.784297,0.565433,0.434567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1557.093688,1575.641044,0.678325,0.321675,NaN,NaN,76,29,53
1226,2022-04-09,2022,0,NaN,DEN,LAL,1540.948928,1481.797129,0.714257,0.285743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1596.551861,1521.953823,0.821170,0.178830,NaN,NaN,73,28,51
1227,2022-04-09,2022,0,NaN,PHO,SAC,1662.260235,1364.967937,0.907795,0.092205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1634.712133,1421.126858,0.859519,0.140481,NaN,NaN,59,3,31
1228,2022-04-09,2022,0,NaN,NOP,GSW,1449.249520,1655.076359,0.352244,0.647756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1520.080842,1612.149058,0.501471,0.498529,NaN,NaN,76,12,44


**Predict teams that will qualify for play offs**

In [ ]:
# Determine whether the home team won
test_df['Home Win'] = test_df['Visitor Score'] < test_df['Home Score']

# Assign "class values" to the Home Win column
y_true = test_df['Home Win'].values

**View the new column**

In [ ]:
test_df

,Date,Season,neutral,playoff,Visitor Team,Home Team,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,Visitor Score,Home Score,quality,importance,total_rating,Home Win
0,2021-10-19,2022,0,NaN,MIL,BRK,1657.749890,1605.053732,0.706613,0.293387,1667.197782,1595.605840,NaN,NaN,NaN,NaN,NaN,NaN,1659.743630,1613.934913,0.700783,0.299217,127.0,104.0,92,14,53,False
1,2021-10-19,2022,0,NaN,LAL,GSW,1549.258851,1529.010908,0.666457,0.333543,1536.851828,1541.417931,NaN,NaN,NaN,NaN,NaN,NaN,1510.584522,1530.407615,0.608488,0.391512,114.0,121.0,59,54,57,True
2,2021-10-20,2022,0,NaN,CHO,IND,1438.333792,1487.419833,0.572751,0.427249,1441.652424,1484.101200,NaN,NaN,NaN,NaN,NaN,NaN,1457.196370,1532.967268,0.536682,0.463318,123.0,122.0,48,51,50,False
3,2021-10-20,2022,0,NaN,DET,CHI,1380.825861,1495.430593,0.478994,0.521006,1373.503545,1502.752909,NaN,NaN,NaN,NaN,NaN,NaN,1369.777383,1512.209592,0.442543,0.557457,88.0,94.0,26,19,23,True
4,2021-10-20,2022,0,NaN,TOR,WAS,1449.348147,1494.785854,0.577882,0.422118,1433.077337,1511.056663,NaN,NaN,NaN,NaN,NaN,NaN,1461.247869,1525.861517,0.546017,0.453983,83.0,98.0,48,66,57,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,2022-04-09,2022,0,NaN,MIN,CHI,1525.513746,1579.784297,0.565433,0.434567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1557.093688,1575.641044,0.678325,0.321675,NaN,NaN,76,29,53,False
1226,2022-04-09,2022,0,NaN,DEN,LAL,1540.948928,1481.797129,0.714257,0.285743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1596.551861,1521.953823,0.821170,0.178830,NaN,NaN,73,28,51,False
1227,2022-04-09,2022,0,NaN,PHO,SAC,1662.260235,1364.967937,0.907795,0.092205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1634.712133,1421.126858,0.859519,0.140481,NaN,NaN,59,3,31,False
1228,2022-04-09,2022,0,NaN,NOP,GSW,1449.249520,1655.076359,0.352244,0.647756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1520.080842,1612.149058,0.501471,0.498529,NaN,NaN,76,12,44,False


In [ ]:
# Create two new columns, with all of the values set to False
test_df['Home Last Win'] = False
test_df['Visitor Last Win'] = False

# Determining whether the home and visitor teams won their last games
# This will update the Home Last Win & Visitor Last Win columns to either remain False or change to True

from collections import defaultdict

won_last = defaultdict(int)   # The default value of int is 0

# Inefficient method to deterime previous wins
for index, row in test_df.iterrows():
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    row['Home Last Win'] = won_last[home_team]
    row['Visitor Last Win'] = won_last[visitor_team]
    # Set current win
    won_last[home_team] = row['Home Win']
    won_last[visitor_team] = not row['Home Win']
test_df.sample(10)

,Date,Season,neutral,playoff,Visitor Team,Home Team,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,Visitor Score,Home Score,quality,importance,total_rating,Home Win,Home Last Win,Visitor Last Win
180,2021-11-12,2022,0,NaN,SAS,DAL,1480.785274,1527.307502,0.576359,0.423641,1465.296755,1542.796021,NaN,NaN,NaN,NaN,NaN,NaN,1458.179575,1561.306468,0.501827,0.498173,109.0,123.0,54,46,50,True,False,False
919,2022-02-28,2022,0,NaN,MIL,CHO,1587.572303,1509.602537,0.735844,0.264156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1636.279784,1498.147880,0.838321,0.161679,NaN,NaN,75,22,49,False,False,False
1000,2022-03-11,2022,0,NaN,SAS,UTA,1498.964811,1662.326452,0.409813,0.590187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493.796130,1684.872815,0.378116,0.621884,NaN,NaN,81,28,55,False,False,False
12,2021-10-20,2022,0,NaN,PHO,DEN,1650.231580,1577.362511,0.730097,0.269903,1630.513539,1597.080552,NaN,NaN,NaN,NaN,NaN,NaN,1651.385410,1594.785883,0.717771,0.282229,98.0,110.0,90,10,50,True,False,False
359,2021-12-06,2022,0,NaN,PHO,SAS,1694.708907,1503.344526,0.842537,0.157463,1696.324121,1501.729312,NaN,NaN,NaN,NaN,NaN,NaN,1644.703398,1489.659761,0.814499,0.185501,108.0,104.0,75,14,45,False,False,False
705,2022-01-25,2022,0,NaN,PHI,NOP,1604.486530,1449.249520,0.812945,0.187055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1622.658855,1455.878005,0.851082,0.148918,NaN,NaN,65,6,36,False,False,False
439,2021-12-18,2022,0,NaN,BOS,NYK,1531.836113,1486.549855,0.697693,0.302307,1536.392955,1481.993012,NaN,NaN,NaN,NaN,NaN,NaN,1593.745537,1478.570542,0.728057,0.271943,114.0,107.0,65,38,52,False,False,False
1122,2022-03-28,2022,0,NaN,CLE,ORL,1564.871230,1289.369869,0.896744,0.103256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1505.202062,1334.258402,0.829347,0.170653,NaN,NaN,19,15,17,False,False,False
1213,2022-04-08,2022,0,NaN,SAS,GSW,1498.964811,1655.076359,0.419944,0.580056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493.796130,1612.149058,0.473936,0.526064,NaN,NaN,71,20,46,False,False,False
743,2022-01-30,2022,0,NaN,ATL,LAL,1480.785635,1481.797129,0.638722,0.361278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1571.620459,1521.953823,0.706341,0.293659,NaN,NaN,69,57,63,False,False,False


In [ ]:
# Creating columns for winning streaks
test_df["Home Win Streak"] = 0
test_df["Visitor Win Streak"] = 0

# Did the home and visitor teams win their last game?

from collections import defaultdict

win_streak = defaultdict(int)

for index, row in test_df.iterrows():  # Note that this is not the most efficient method
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["Home Win Streak"] = win_streak[home_team]
    row["Visitor Win Streak"] = win_streak[visitor_team]
    test_df.loc[index] = row    
    
    # Set current win streak
    if row["Home Win"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1
        
test_df.sample(10)

,Date,Season,neutral,playoff,Visitor Team,Home Team,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,Visitor Score,Home Score,quality,importance,total_rating,Home Win,Home Last Win,Visitor Last Win,Home Win Streak,Visitor Win Streak
1228,2022-04-09,2022,0,NaN,NOP,GSW,1449.249520,1655.076359,0.352244,0.647756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1520.080842,1612.149058,0.501471,0.498529,NaN,NaN,76,12,44,False,False,False,0,0
348,2021-12-05,2022,0,NaN,CLE,UTA,1516.066872,1642.760716,0.461660,0.538340,1512.412926,1646.414662,NaN,NaN,NaN,NaN,NaN,NaN,1489.113190,1659.693910,0.408030,0.591970,108.0,109.0,77,25,51,True,False,False,3,4
68,2021-10-28,2022,0,NaN,DAL,SAS,1542.371510,1465.210671,0.734937,0.265063,1545.639089,1461.943092,NaN,NaN,NaN,NaN,NaN,NaN,1584.522775,1494.653306,0.746305,0.253695,104.0,99.0,67,23,45,False,False,False,0,2
831,2022-02-11,2022,0,NaN,PHI,OKC,1604.486530,1314.638576,0.904144,0.095856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1622.658855,1290.000297,0.923496,0.076504,NaN,NaN,26,0,13,False,False,False,2,1
1101,2022-03-25,2022,0,NaN,MIN,DAL,1525.513746,1581.837625,0.562526,0.437474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1557.093688,1607.324621,0.578277,0.421723,NaN,NaN,80,40,60,False,False,False,2,1
702,2022-01-24,2022,0,NaN,NOP,IND,1449.249520,1477.944900,0.601199,0.398801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1455.878005,1543.321698,0.508103,0.491897,NaN,NaN,50,28,39,False,False,False,0,0
232,2021-11-19,2022,0,NaN,BRK,ORL,1590.472018,1332.340047,0.887112,0.112888,1591.319990,1331.492075,NaN,NaN,NaN,NaN,NaN,NaN,1603.267960,1336.703209,0.887731,0.112269,115.0,113.0,33,2,18,False,False,False,1,1
1001,2022-03-11,2022,0,NaN,PHO,TOR,1662.260235,1546.477207,0.775938,0.224062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1646.968523,1567.371834,0.732132,0.267868,NaN,NaN,87,29,58,False,False,False,0,0
444,2021-12-18,2022,0,NaN,MIL,CLE,1622.263106,1559.829931,0.718097,0.281903,1587.048263,1595.044774,NaN,NaN,NaN,NaN,NaN,NaN,1460.487439,1523.192946,0.478577,0.521423,90.0,119.0,47,39,43,True,False,False,5,0
1145,2022-03-31,2022,0,NaN,DET,PHI,1314.172351,1604.486530,0.250577,0.749423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1310.987364,1622.658855,0.228893,0.771107,NaN,NaN,30,2,16,False,False,False,1,0


In [ ]:
# Build function to add Home Team Ranks Higher feature
def home_team_ranks_higher_2015(row):
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    
    # Adjusting the Charlotte team names due to off-season league changes between 13-14 & 14-15
    if home_team == "WAS":
        home_team = "Washington wizards"
    if visitor_team == "MEM":
        visitor_team = "Memphis guys"
    
    home_rank = test_df.loc[home_team]['Rank']
    visitor_rank = test_df.loc[visitor_team]['Rank']
    
    # The higher ranking will be the lower number
    return home_rank < visitor_rank

In [ ]:
last_match_winner = defaultdict(int)
test_df["Home Team Won Last"] = 0

for index, row in test_df.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    teams = tuple(sorted([home_team, visitor_team]))  # Sort for a consistent ordering
    
    # Set in the row, who won the last encounter
    row["Home Team Won Last"] = 1 if last_match_winner[teams] == row["Home Team"] else 0
    test_df.loc[index] = row
    
    # Who won this one?
    winner = row["Home Team"] if row["Home Win"] else row["Visitor Team"]
    last_match_winner[teams] = winner
    
test_df.sample(7)

,Date,Season,neutral,playoff,Visitor Team,Home Team,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,Visitor Score,Home Score,quality,importance,total_rating,Home Win,Home Last Win,Visitor Last Win,Home Win Streak,Visitor Win Streak,Home Team Won Last
518,2021-12-31,2022,0,NaN,IND,CHI,1484.770168,1575.631967,0.513148,0.486852,1479.774726,1580.627409,NaN,NaN,NaN,NaN,NaN,NaN,1469.242151,1475.076638,0.631730,0.368270,106.0,108.0,39,33,36,True,False,False,5,0,1
968,2022-03-07,2022,0,NaN,MIN,POR,1525.513746,1418.842509,0.766687,0.233313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1557.093688,1536.431815,0.666725,0.333275,NaN,NaN,69,76,73,False,False,False,0,1,0
119,2021-11-04,2022,0,NaN,ATL,UTA,1556.464100,1648.554008,0.511382,0.488618,1540.787704,1664.230404,NaN,NaN,NaN,NaN,NaN,NaN,1593.934333,1657.054135,0.495971,0.504029,98.0,116.0,90,19,55,True,False,False,2,0,0
822,2022-02-09,2022,0,NaN,UTA,GSW,1662.326452,1655.076359,0.649623,0.350377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1684.872815,1612.149058,0.768648,0.231352,NaN,NaN,94,6,50,False,False,False,0,3,1
798,2022-02-06,2022,0,NaN,ORL,BOS,1289.369869,1530.183501,0.307765,0.692235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1334.258402,1589.040384,0.238401,0.761599,NaN,NaN,30,17,24,False,False,False,0,1,1
313,2021-11-30,2022,0,NaN,BRK,NYK,1606.129913,1519.988554,0.744885,0.255115,1608.275734,1517.842733,NaN,NaN,NaN,NaN,NaN,NaN,1602.374364,1503.870488,0.755178,0.244822,112.0,110.0,71,24,48,False,False,False,1,0,0
1180,2022-04-05,2022,0,NaN,TOR,ATL,1546.477207,1480.785635,0.721878,0.278122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1567.371834,1571.620459,0.633117,0.366883,NaN,NaN,77,91,84,False,False,False,2,1,1


**Review the added features in the 2021-2022 season**

In [ ]:
# Review the added features in the 2021-2022 season DataFrame
test_df[['Date', 'Visitor Team', 'Visitor Score', 'Home Team', 'Home Score', 'Home Win', 'Home Last Win', 
       'Visitor Last Win', 'Home Win Streak', 'Visitor Win Streak', 
       'Home Team Won Last']][:5]

,Date,Visitor Team,Visitor Score,Home Team,Home Score,Home Win,Home Last Win,Visitor Last Win,Home Win Streak,Visitor Win Streak,Home Team Won Last
0,2021-10-19,MIL,127.0,BRK,104.0,False,False,False,0,0,0
1,2021-10-19,LAL,114.0,GSW,121.0,True,False,False,0,0,0
2,2021-10-20,CHO,123.0,IND,122.0,False,False,False,0,0,0
3,2021-10-20,DET,88.0,CHI,94.0,True,False,False,0,0,0
4,2021-10-20,TOR,83.0,WAS,98.0,True,False,False,0,0,0


In [ ]:
test_df['Label'] = [1 if x > 0 else 0 for x in test_df['Home Score'] - test_df['Visitor Score']]
test_df.drop(columns=['Home Score', 'Visitor Score'], inplace=True)
final_team_stats = test_df.dropna()

combined_copy = final_team_stats

final_team_stats

,Date,Season,neutral,playoff,Visitor Team,Home Team,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,quality,importance,total_rating,Home Win,Home Last Win,Visitor Last Win,Home Win Streak,Visitor Win Streak,Home Team Won Last,Label


In [ ]:
test_df[['Date', 'Season', 'Visitor Team', 'Home Team',  'Home Win', 'Home Win Streak', 'Visitor Win Streak'
       ]][:20]

,Date,Season,Visitor Team,Home Team,Home Win,Home Win Streak,Visitor Win Streak
0,2021-10-19,2022,MIL,BRK,False,0,0
1,2021-10-19,2022,LAL,GSW,True,0,0
2,2021-10-20,2022,CHO,IND,False,0,0
3,2021-10-20,2022,DET,CHI,True,0,0
4,2021-10-20,2022,TOR,WAS,True,0,0
5,2021-10-20,2022,NYK,BOS,False,0,0
6,2021-10-20,2022,MEM,CLE,False,0,0
7,2021-10-20,2022,NOP,PHI,True,0,0
8,2021-10-20,2022,MIN,HOU,False,0,0
9,2021-10-20,2022,SAS,ORL,False,0,0


,Date,Season,neutral,playoff,Visitor Team,Home Team,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,quality,importance,total_rating,Home Win,Home Last Win,Visitor Last Win,Home Win Streak,Visitor Win Streak,Home Team Won Last,Label
1,2021-10-19,2022,0,NaN,LAL,GSW,1549.258851,1529.010908,0.666457,0.333543,1536.851828,1541.417931,NaN,NaN,NaN,NaN,NaN,NaN,1510.584522,1530.407615,0.608488,0.391512,59,54,57,True,False,False,0,0,0,1
3,2021-10-20,2022,0,NaN,DET,CHI,1380.825861,1495.430593,0.478994,0.521006,1373.503545,1502.752909,NaN,NaN,NaN,NaN,NaN,NaN,1369.777383,1512.209592,0.442543,0.557457,26,19,23,True,False,False,0,0,0,1
4,2021-10-20,2022,0,NaN,TOR,WAS,1449.348147,1494.785854,0.577882,0.422118,1433.077337,1511.056663,NaN,NaN,NaN,NaN,NaN,NaN,1461.247869,1525.861517,0.546017,0.453983,48,66,57,True,False,False,0,0,0,1
7,2021-10-20,2022,0,NaN,NOP,PHI,1482.179835,1609.071634,0.461377,0.538623,1467.383214,1623.868256,NaN,NaN,NaN,NaN,NaN,NaN,1529.566470,1654.186224,0.462987,0.537013,83,18,51,True,False,False,0,0,0,1
11,2021-10-20,2022,0,NaN,POR,SAC,1581.015607,1454.318618,0.786671,0.213329,1570.271051,1465.063173,NaN,NaN,NaN,NaN,NaN,NaN,1619.912596,1453.118951,0.819450,0.180550,64,14,39,True,False,False,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,2022-01-08,2022,0,NaN,PHO,MIA,1697.880204,1559.508299,0.797732,0.202268,1662.260235,1595.128268,NaN,NaN,NaN,NaN,NaN,NaN,1679.382642,1512.641362,0.829526,0.170474,83,5,44,True,False,False,1,3,0,1
588,2022-01-09,2022,0,NaN,ORL,WAS,1293.043561,1462.546785,0.401290,0.598710,1289.369869,1466.220477,NaN,NaN,NaN,NaN,NaN,NaN,1309.118325,1472.349947,0.338727,0.661273,11,10,11,True,False,False,0,0,1,1
589,2022-01-09,2022,0,NaN,OKC,DEN,1318.517100,1537.070404,0.335715,0.664285,1314.638576,1540.948928,NaN,NaN,NaN,NaN,NaN,NaN,1307.237720,1543.151648,0.321418,0.678582,19,4,12,True,False,False,1,0,0,1
590,2022-01-09,2022,0,NaN,HOU,MIN,1314.443156,1515.405067,0.358660,0.641340,1304.334477,1525.513746,NaN,NaN,NaN,NaN,NaN,NaN,1301.467668,1541.423215,0.302876,0.697124,18,16,17,True,False,False,3,0,1,1


In [ ]:
test_df.loc[90:95][['Date', 'elo1_pre', 'elo2_pre', 'Home Team', 'Home Win Streak', 
               'Home Win',]]

,Date,elo1_pre,elo2_pre,Home Team,Home Win Streak,Home Win
90,2021-10-31,1613.657227,1631.139368,UTA,0,True
91,2021-10-31,1568.318588,1368.015154,DET,1,False
92,2021-10-31,1527.656476,1330.900837,HOU,0,False
93,2021-11-01,1619.857289,1583.580349,POR,0,False
94,2021-11-01,1481.003052,1401.886700,CLE,0,True
95,2021-11-01,1459.901546,1482.537738,SAS,1,False


Maximum number in the test dataset

In [ ]:
max(test_df)

'total_rating'

In [ ]:
max(test_df['total_rating'])

85

**The team most likely to win the 2022 NBA Finals based on total ratings **

In [ ]:
df[df['total_rating']>84]

,Date,Season,neutral,playoff,Visitor Team,Home Team,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,Visitor Score,Home Score,quality,importance,total_rating,Home Win,Home Win Streak,Visitor Win Streak
71642,2022-03-01,2022,0,NaN,BOS,ATL,1530.183501,1480.785635,0.702661,0.297339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1589.040384,1571.620459,0.653881,0.346119,NaN,NaN,80,90.0,85.0,False,1,0


Based on total ratings, the team most likely to win the 2022 NBA Championship are the Boston celtics